In [ ]:
# Remove input cells at runtime (nbsphinx)
import IPython.core.display as d
d.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

# Image cleaning

**Author(s):** Dr. Michele Peresano (CEA-Saclay/IRFU/DAp/LEPCHE), 2020

**Recommended datasample(s):** ``gamma1`` (dataset used to build the energy model)

**Data level(s):** DL1b (telescope-wise image parameters)

**Description:**

This notebook contains benchmarks and metrics from the _protopipe_ pipeline aimed at the DL1b data level (cleaned and parametrized images).  

**Requirements and steps to reproduce:**

To run this notebook you will need a TRAINING file generated using ``protopipe-TRAINING`` .

To get a filled notebook and reproduce these results,

- get the necessary input files using ``protopipe-TRAINING``
- execute the notebook with ``protopipe-BENCHMARK``

``protopipe-BENCHMARK launch --config_file configs/benchmarks.yaml -n TRAINING/benchmarks_DL1_image-cleaning``

To obtain the list of all available parameters add ``--help-notebook``.

**Comparison against CTAMARS:**

- the input file needs to be a merged TRAINING file from the ``gamma1`` sample,
- reference simtel-files, plots, values and settings can be found [here (please, always refer to the latest version)](https://forge.in2p3.fr/projects/benchmarks-reference-analysis/wiki/Comparisons_between_pipelines).

**Development and testing:**  

As with any other part of _protopipe_ and being part of the official repository, this notebook can be further developed by any interested contributor.  
The execution of this notebook is not currently automatic, it must be done locally by the user _before_ pushing a pull-request.
Please, strip the output before pushing.

## Table of contents
  - [Fraction of events (relative to telescope triggers) that survive a given intensity cut](#Fraction-of-events-(relative-to-telescope-triggers)-that-survive-a-given-intensity-cut)
  - [Image-parameter distributions](#Image-parameter-distributions)
      + [LST-1 image intensity](#LST-1-image-intensity)
      + [DL1 Parameters used for energy training from all telecopes](#DL1-Parameters-used-for-energy-training-from-all-telecopes)

## Imports

In [ ]:
import os
from pathlib import Path
import warnings

def fxn():
    warnings.warn("runtime", RuntimeWarning)

import tables
import numpy as np
import pandas
import uproot
from scipy.stats import binned_statistic, binned_statistic_2d, cumfreq, percentileofscore
from astropy import units as u
from astropy.table import Table
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from mpl_toolkits.mplot3d import Axes3D

from protopipe.pipeline.io import get_camera_names, read_protopipe_TRAINING_per_tel_type
from protopipe.pipeline.utils import add_stats, CTAMARS_radii

## Input data
[back to top](#Table-of-contents)

### Protopipe
[back to top](#Table-of-contents)

In [ ]:
# Parametrized cell
# Modify these variables according to your local setup outside of the Vagrant Box
analyses_directory = "/Users/michele/Applications/ctasoft/dirac/shared_folder/analyses/"  # path to the 'analyses' analyses folder
output_directory = Path.cwd() # default output directory for plots
analysis_name = "test_from_docker_2" # Name of the analysis stored in 'analyses_folder'
load_CTAMARS_2019 = True  # If True (default), compare to the CTAN analysis done with CTAMARS (Release 2019)
input_filename = None # Name of the file produced with protopipe
CTAMARS_put_directory = None # Path to DL1 CTAMARS data (if load_CTAMARS_2019 is True)
min_pixels = 3
min_ellipticity = 0.1
max_ellipticity = 0.6
containment_radius = 0.8 # from 0 to 1 (whole camera)
protopipe_intensity_cut = 50 # phe

In [ ]:
input_directory = Path(analyses_directory) / analysis_name / Path("data/TRAINING/for_energy_estimation")

if not input_filename:
    try:
        input_filename = input_filenames["TRAINING_energy_gamma"]
    except (NameError, KeyError):
        input_filename = "TRAINING_energy_tail_gamma_merged.h5"

cameras = get_camera_names(input_directory = input_directory,
                           file_name = input_filename)
data = read_protopipe_TRAINING_per_tel_type(input_directory = input_directory,
                                            file_name = input_filename,
                                            camera_names=cameras)

In [ ]:
selected_data = {}

if load_CTAMARS_2019:

    # Remove from protopipe's data images that did not survive the preliminary image cleaning
    # between the 2 image extraction passes
    for camera in cameras:
        selected_data[camera] = data[camera].query("image_extraction == 1")
        
else:
    
    for camera in cameras:
        selected_data[camera] = data[camera]

### CTA-MARS
[back to top](#Table-of-contents)

In [ ]:
if load_CTAMARS_2019:
    
    input_directory_CTAMARS = {}
    input_directory_CTAMARS["parent_directory"] = "/Users/michele/Applications/ctasoft/tests/CTAMARS_reference_data"
    input_directory_CTAMARS["TRAINING/DL1"] = "TRAINING/DL1"
    
    # Get input file path
    if (input_directory_CTAMARS["parent_directory"] is None) or (input_directory_CTAMARS["TRAINING/DL1"] is None):
        raise ValueError("ERROR: CTAMARS data undefined. Please, check the documentation of protopipe-BENCHMARKS.")
    else:

        # read CTAMARS ROOT files
        mars_dl1b_fileName = "check_dl1b.root"
        path_mars_dl1b = Path(input_directory_CTAMARS["parent_directory"]) / input_directory_CTAMARS["TRAINING/DL1"] / mars_dl1b_fileName
        ctamars_dl1b = uproot.open(path_mars_dl1b)

        mars_LST1size_fileName = "LST1_SIZE_distro_gamma1sample.root"
        path_mars_LST1size = Path(input_directory_CTAMARS["parent_directory"]) / input_directory_CTAMARS["TRAINING/DL1"] /  mars_LST1size_fileName
        ctamars_LST1size = uproot.open(path_mars_LST1size)

        # create histograms
        mars_size_npixels_LSTCam = ctamars_dl1b["log10Size_type0"].to_numpy()
        mars_size_npixels_NectarCam = ctamars_dl1b["log10Size_type1"].to_numpy()
        mars_size_WL_LSTCam = ctamars_dl1b["log10Size_WL_type0"].to_numpy()
        mars_size_WL_NectarCam = ctamars_dl1b["log10Size_WL_type1"].to_numpy()
        mars_size_d80_LSTCam = ctamars_dl1b["log10Size_d80_type0"].to_numpy()
        mars_size_d80_NectarCam = ctamars_dl1b["log10Size_d80_type1"].to_numpy()
        mars_size_LST1Cam = ctamars_LST1size["h"].to_numpy()

        # fill camera-wise dictionaries
        CTAMARS = {}
        CTAMARS["LSTCam"] = {"size_npixels": mars_size_npixels_LSTCam, 
                             "size_WL" : mars_size_WL_LSTCam, 
                             "size_d80" : mars_size_d80_LSTCam, 
                             "size_LST1" : mars_size_LST1Cam}
        CTAMARS["NectarCam"] = {"size_npixels": mars_size_npixels_NectarCam, 
                                "size_WL" : mars_size_WL_NectarCam, 
                                "size_d80" : mars_size_d80_NectarCam}

## Plots and benchmarks
[back to top](#Table-of-contents)

In [ ]:
# First we check if a _plots_ folder exists already.  
# If not, we create it.
plots_folder = Path(output_directory) / "plots"
plots_folder.mkdir(parents=True, exist_ok=True)

### Fraction of events (relative to telescope triggers) that survive a given intensity cut
[back to top](#Table-of-contents)

Multi-cluster cleaning

If the "no-cuts" curve doesn't start at 1, it's because some images were so bad that they couldn't get a valid parametrization and have been recorded with ``hillas_intensity = NaN``.

In [ ]:
for camera in cameras:
    
    fig = plt.figure(figsize=(6, 5), tight_layout=False)
    plt.xlabel("log10(intensity #p.e)")
    plt.ylabel("Telescope triggers fraction with log10(intensity #p.e) > x phe")
    plt.title(camera)

    tot_entries = len(data[camera]["hillas_intensity"])
    
    xbins = CTAMARS[camera]["size_WL"][1]

    # No cuts
    selected_images = data[camera]
    intensity_hist, xbins = np.histogram(np.log10(selected_images["hillas_intensity"]), bins=xbins)
    plt.plot(xbins[:-1], intensity_hist[::-1].cumsum()[::-1]/tot_entries, drawstyle="steps-post", label="No cuts", color="steelblue")
    
    # Cut in the number of pixels
    selected_images = selected_data[camera].query(f"pixels > {min_pixels}")
    intensity_hist, xbins = np.histogram( np.log10(selected_images["hillas_intensity"]), bins=xbins)
    plt.plot(xbins[:-1], intensity_hist[::-1].cumsum()[::-1]/tot_entries, drawstyle="steps-post", label="+ n_pixel", color="orange")
    
    # Cut in ellipticity
    selected_images = selected_data[camera].query(f"pixels > {min_pixels}\
                                           and hillas_ellipticity > {min_ellipticity}\
                                           and hillas_ellipticity < {max_ellipticity}")
    intensity_hist, xbins = np.histogram( np.log10(selected_images["hillas_intensity"]), bins=xbins)
    plt.plot(xbins[:-1], intensity_hist[::-1].cumsum()[::-1]/tot_entries, drawstyle="steps-post", label="+ ellipticity", color="green")
    
    # Cut in containment radius
    selected_images = selected_data[camera].query(f"pixels > {min_pixels}\
                                           and hillas_ellipticity > {min_ellipticity}\
                                           and hillas_ellipticity < {max_ellipticity}\
                                           and hillas_r < {(CTAMARS_radii(camera)*containment_radius)}")
    intensity_hist, xbins = np.histogram( np.log10(selected_images["hillas_intensity"]), bins=xbins)
    plt.plot(xbins[:-1], intensity_hist[::-1].cumsum()[::-1]/tot_entries, drawstyle="steps-post", label="+ COG continment", color="red")
    
    # Plot CTAMARS data

    x = 0.5 * (CTAMARS[camera]["size_WL"][1][1:] + CTAMARS[camera]["size_WL"][1][:-1])
    plt.step(x, CTAMARS[camera]["size_npixels"][0], where='mid', label='CTAMARS npixels', color="orange", linestyle="--")
    plt.step(x, CTAMARS[camera]["size_WL"][0], where='mid', label='+ CTAMARS ellipticity', color="green", linestyle="--")
    plt.step(x, CTAMARS[camera]["size_d80"][0], where='mid', label='+ CTAMARS COG continment', color="red", linestyle="--")
    
    plt.ylim([0.,1.05])
    ax = plt.gca()
    ylims=ax.get_ylim()

    CTAMARS_intensity_cut = 50
    plt.vlines(np.log10(CTAMARS_intensity_cut),
               ymin=min(ylims), ymax=max(ylims),
               ls="dashed", lw=2,
               color="darkorange",
               label=f"{CTAMARS_intensity_cut} phe (CTAMARS)")

    plt.vlines(np.log10(protopipe_intensity_cut),
               ymin=min(ylims), ymax=max(ylims),
               ls="dashed", lw=2,
               color="blue",
               label=f"{protopipe_intensity_cut} phe (protopipe)")
    
    plt.minorticks_on()
    plt.grid()
    plt.legend()

    fig.savefig(plots_folder / f"image_cleaning_eventsAboveIntensity_{camera}_protopipe_{analysis_name}.png")
    
    plt.show()

### Image-parameter distributions
[back to top](#Table-of-contents)

**Notes**  
 - probably better to make bins in true energy
 - the parameters should be at least those that enter the estimators training (here only the pure DL1 are listed)

#### LST-1 image intensity
[back to top](#Table-of-contents)

Here we compare the distribution of image insensities from LST-1 from the gamma1 sample.
Therefore the key used for intensity is the one used for shower geometry reconstruction.

In [ ]:
if load_CTAMARS_2019:
    size_LST1 = data["LSTCam"].query("tel_id == 1 and image_extraction == 1")["hillas_intensity"]
else:
    size_LST1 = data["LSTCam"].query("tel_id == 1")["hillas_intensity"]

x_bins_edges = CTAMARS["LSTCam"]["size_LST1"][1]
CTAMARS_counts = CTAMARS["LSTCam"]["size_LST1"][0]

fig = plt.figure(figsize=(16, 5), tight_layout=False)

plt.subplot(1,2,1)

plt.xlabel(f"log10(hillas_intensity) [#phe]")
plt.ylabel("Number of images")
plt.title("LST1 - gamma1")
h_protopipe = plt.hist(np.log10(size_LST1),
                       bins=x_bins_edges,
                       histtype="step",
                       label="protopipe", color="blue")

print(f"Total number of images for CTAMARS = {np.sum(CTAMARS_counts)}")
print(f"Total number of images for protopipe = {np.sum(h_protopipe[0])}")


plt.step(x_bins_edges[:-1], CTAMARS_counts, where='pre', label='CTAMARS', color="darkorange")

plt.yscale('log')
plt.minorticks_on()
plt.grid(which = "both")

ax = plt.gca()
ylims=ax.get_ylim()

plt.vlines(np.log10(CTAMARS_intensity_cut),
           ymin=min(ylims), ymax=max(ylims),
           ls="dashed", lw=2,
           color="darkorange",
           label=f"{CTAMARS_intensity_cut} phe (CTAMARS)")

plt.vlines(np.log10(protopipe_intensity_cut),
           ymin=min(ylims), ymax=max(ylims),
           ls="dashed", lw=2,
           color="blue",
           label=f"{protopipe_intensity_cut} phe (protopipe)")

plt.legend()

plt.ylim(1, 1.e5)

plt.subplot(1,2,2)

plt.xlabel(f"log10(hillas_intensity) [#phe]")
plt.ylabel(f"Ratio protopipe / CTAMARS")
x = 0.5 * (x_bins_edges[1:] + x_bins_edges[:-1])

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()
    plt.step(x, h_protopipe[0]/CTAMARS_counts, where='pre')

ax = plt.gca()
xlims=ax.get_xlim()
xlims=[np.min(x_bins_edges),np.max(x_bins_edges)]
plt.hlines(1., xlims[0], xlims[1], label="expectation", color='r')
plt.grid()
plt.legend()
plt.ylim(0, 3)

fig.savefig(plots_folder / f"image_cleaning_hillas_intensity_LST1_gamma1_{camera}_protopipe_{analysis_name}.png")

plt.show()

#### DL1 Parameters used for direction reconstruction from all telecopes
[back to top](#Table-of-contents)

In [ ]:
nbins = 100
parameters_to_plot = ["hillas_intensity", "hillas_width", "hillas_length"]

fig, axes = plt.subplots(ncols=len(parameters_to_plot), 
                         nrows=len(cameras), 
                         constrained_layout=False, 
                         figsize = (40, 20))
plt.subplots_adjust(hspace = 0.5)
fontsize=20

for i, camera in enumerate(cameras):
        
    for j, key in enumerate(parameters_to_plot):
        
        axes[i, j].set_ylabel("Number of events", fontsize=fontsize)
        axes[i, j].set_title(camera, fontsize=fontsize)

        if "hillas_intensity" in key:
            axes[i, j].set_xlabel(f"log10({key}) [#phe]", fontsize=fontsize)
            axes[i, j].hist(np.log10(data[camera][key]), 
                            bins=nbins, 
                            range=[1.,6.], 
                            alpha = 0.5, 
                            histtype="step",
                            linewidth=5)
            add_stats(np.log10(data[camera][key]), axes[i, j], x=0.70, y=0.85, fontsize=fontsize)
        else:
            axes[i, j].set_xlabel(f"{key} [deg]", fontsize=fontsize)
            axes[i, j].hist(data[camera][key], 
                            bins=nbins, 
                            alpha = 0.5, 
                            histtype="step",
                            linewidth=5)
            add_stats(data[camera][key], axes[i, j], x=0.70, y=0.85, fontsize=fontsize)
            

        axes[i, j].set_yscale('log')
        axes[i, j].minorticks_on()
        axes[i, j].grid(which = "both")
        
        # Save just the portion _inside_ the second axis's boundaries
        extent = axes[i, j].get_window_extent().transformed(fig.dpi_scale_trans.inverted())
        fig.savefig(plots_folder / f"image_cleaning_{key}_gamma1_{camera}_protopipe_{analysis_name}.png", bbox_inches=extent.expanded(1.2, 1.2))

fig.savefig(plots_folder / f"image_cleaning_gamma1_allKeysallCameras_protopipe_{analysis_name}.png")

plt.show()